In [ ]:
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 4.9 MB/s e

## Template 사용해보기

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 'KEY'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'ENDPOINT'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [ ]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

In [ ]:
llm = AzureOpenAI(
    deployment_name = 'dev-gpt-35-turbo-instruct',

    # 제한을 걸어주어야 토큰 낭비가 되지 않는다.
    # 만약 제한이 없다면 사용할 수 있는 토큰에서 계속 떠들고 있을
    max_tokens = 1000
)

In [ ]:
# 템플릿 정의하기
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# 기본 query 정의
string_prompt = PromptTemplate.from_template('Tell me a joke about {subject}')

# 내가 어떤 주제를 넘기느냐에 따라 다양한 질문을 만들 수 있다.
string_prompt_value = string_prompt.format_prompt(subject = 'soccer')

# 질문 확인
string_prompt_value.text

'Tell me a joke about soccer'

In [ ]:
template = '''
너는 요리사야. 내가 가지고 있는 재료들로 만들 수 있는 요리를 추천하고 그 요리의 레시피를 제시해
내가 가진 재료는 아래와 같아

<재료>
{재료}
'''

# {재료}부분을 갈아 낄 수 있도록 만들겠다.
prompt_template = PromptTemplate(
    input_variables=['재료'],

    # 위에서 정의한 템플릿 넣어주
    template = template
)

prompt_template

PromptTemplate(input_variables=['재료'], template='\n너는 요리사야. 내가 가지고 있는 재료들로 만들 수 있는 요리를 추천하고 그 요리의 레시피를 제시해\n내가 가진 재료는 아래와 같아\n\n<재료>\n{재료}\n')

In [ ]:
prompt_template.format(재료='양파, 계란, 당근, 빵')

'\n너는 요리사야. 내가 가지고 있는 재료들로 만들 수 있는 요리를 추천하고 그 요리의 레시피를 제시해\n내가 가진 재료는 아래와 같아\n\n<재료>\n양파, 계란, 당근, 빵\n'

In [ ]:
# 답을 출력해보기
llm(prompt_template.format(재료='양파, 계란, 사과, 빵'))

'\n추천 요리: 양파 계란볶음밥\n\n레시피:\n1. 양파를 채 썰어준다.\n2. 계란을 풀어서 소금과 후추로 간을 한다.\n3. 팬에 식용유를 두르고 양파를 볶아준다.\n4. 양파가 익으면 계란을 넣어서 볶아준다.\n5. 계란이 익으면 빵을 큼직하게 잘라서 넣어준다.\n6. 빵이 바싹 볶아질 때까지 볶아준다.\n7. 사과를 껍질을 벗겨서 채 썰어준 뒤 마지막에 얹어준다.\n8. 약간의 식초를 넣어서 마무리해주면 완성이다.'

## 대화형 모델의 템플릿

In [ ]:
from langchain.prompts import(
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)

from langchain.schema import(
    SystemMessage,
    AIMessage,
    HumanMessage
)

In [ ]:
chatgpt = AzureChatOpenAI(
    deployment_name = 'dev-gpt-35-turbo',
    temperature = 0
)

# 시스템 메세지의 프롬프트 설정
# 이전에 정의한 템플릿을 넣어줌
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# 휴먼 메세지의 프롬프트 설정
human_template = '{재료}'
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# chat prompt template
chat_prompt = ChatPromptTemplate.from_messages(
    messages = [
        system_message_prompt,
        human_message_prompt
    ])

# 대화형 모델에서는 message를 뽑아서 넘겨주어야 동작할 수 있다.
answer = chatgpt(chat_prompt.format_prompt(재료='치킨, 계란, 빵').to_messages())
answer

AIMessage(content='가지고 있는 재료로 만들 수 있는 요리 중 하나는 "치킨 샌드위치"입니다. 아래는 치킨 샌드위치의 레시피입니다.\n\n[치킨 샌드위치 레시피]\n재료:\n- 치킨 가슴살 2조각\n- 계란 2개\n- 빵 4장\n- 상추, 토마토, 양파 (선택적으로 추가 가능)\n\n조리 방법:\n1. 치킨 가슴살을 소금과 후추로 조미료한 후 그릴이나 팬에서 구워주세요. 완전히 익을 때까지 약 5-7분 정도 소요됩니다.\n2. 계란을 끓는 물에 넣고 7-8분 정도 삶아주세요. 그 후 찬물에 담가 식힌 뒤 껍질을 벗겨주세요.\n3. 빵을 깨끗한 표면으로 준비해주세요. 필요에 따라 토스터기나 오븐에서 빵을 따뜻하게 해도 좋습니다.\n4. 빵 한 장 위에 구운 치킨 가슴살을 올려주세요. 그 위에 삶은 계란을 얇게 슬라이스하여 올려주세요.\n5. 상추, 토마토, 양파 등의 추가 재료를 원한다면 올려주세요.\n6. 다른 한 장의 빵으로 샌드위치를 덮어주세요.\n7. 샌드위치를 절반으로 잘라서 서빙하거나, 필요에 따라 추가적인 소스나 드레싱을 곁들여 즐겨주세요.\n\n이렇게 만들어진 치킨 샌드위치는 간편하면서도 맛있는 요리입니다. 즐거운 식사 되세요!')

## Few shot Learning

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        'question' : '아이유로 삼행시를 지어줘',
        'answer' : '''
        아 아이유는
        이 이상하고
        유 유난히 좋다.
        '''
    },

    {
        'question' : '이순신으로 삼행시를 지어줘',
        'answer' : '''
        이 이순신 장군은
        순 순수히 내어줄 것 같았던 조선의 바다를
        신 신의와 충의의 마음으로 지켜냈다.
        '''
    }
]

In [ ]:
# 샘플 template
example_prompt = PromptTemplate(
    # 위 프롬프트에서 포함된 key 요소를 적어주어야 한다.
    input_variables=['question', 'answer'],
    template='Question: {question}\n{answer}'
)

# sample template test
# examples에 명명한 탬플릿이 잘 들어갔는지 확인해보자
# 여기서 넘길 때 탬플릿의 값을 넘기는 것이 아니라 주소를 넘겨주어야한다.
# **을 붙여주면 해당 변수의 주소를 넘겨준다
print(example_prompt.format(**examples[0]))

Question: 아이유로 삼행시를 지어줘

        아 아이유는
        이 이상하고
        유 유난히 좋다.
        


In [ ]:
# fewshot learning을 위한 탬플릿 만들기
prompt = FewShotPromptTemplate(
    # 예시를 지정해주어야한다.
    examples = examples,
    example_prompt = example_prompt,
    suffix = 'Question: {input}',
    # 입력받을 값
    input_variables = ['input']
)

In [ ]:
# fewshot을 거치지 않은 것
llm.invoke('손흥민으로 삼행시를 지어줘')

'\n\n손흥민은 영웅이요\n축구는 그의 천국\n매 경기마다 화려한\n그의 발놀림은 정말 신기해'

In [ ]:
# example 뒤에 input이 붙는 것
# 프롬프트 만들기
prompt.format(input = '손흥민으로 삼행시를 지어줘')

'Question: 아이유로 삼행시를 지어줘\n\n        아 아이유는\n        이 이상하고\n        유 유난히 좋다.\n        \n\nQuestion: 이순신으로 삼행시를 지어줘\n\n        이 이순신 장군은\n        순 순수히 내어줄 것 같았던 조선의 바다를\n        신 신의와 충의의 마음으로 지켜냈다.\n        \n\nQuestion: 손흥민으로 삼행시를 지어줘'

In [ ]:
print(llm(prompt.format(input = '손흥민으로 삼행시를 지어줘')))



        손 손흥민은
        흥 흥미로운 축구의 재능을 갖고
        민 민처럼 열심히 뛰며 나라를 대표한다.
